
# 🎬 Premium Cinematic 3‑Clip Video (Colab / GCP Ready)

This notebook turns **one photo** into a **cinematic 3‑clip video** using a lightweight 2.5D parallax method (depth‑aware warping), plus **zoom, pan, color grading, vignetting, lens glow, and slow‑motion**.  
It works on **free Colab GPUs** and also scales great on **Google Cloud A100**.

> If you specifically want *full body motion* driven by big AI models, use Runway/Pika (paid) or gated research models. This notebook focuses on a dependable, premium-looking result without gated model hurdles.

---

## What you get
- **Clip 1 – Intro (Wide → gentle zoom‑in)**  
- **Clip 2 – Mid (parallax pan + lens glow)**  
- **Clip 3 – Close (slow zoom‑in + bokeh‑like blur + fade‑to‑black)**

You'll upload one image and receive `cinematic_video.mp4`.


In [ ]:

#@title ⬇️ Install dependencies (OpenCV, MoviePy, Depth model utils)
!pip install -q opencv-python imageio[ffmpeg] moviepy timm torch torchvision --upgrade

import os, math, cv2, numpy as np, torch, imageio
from moviepy.editor import ImageSequenceClip, vfx, AudioFileClip, concatenate_videoclips
from PIL import Image

# MiDaS depth utils (DPT_Hybrid) from torch.hub
import torch.hub
midas = None
transform = None

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Device:", device)

def load_midas():
    global midas, transform
    if midas is None:
        midas = torch.hub.load("intel-isl/MiDaS", "DPT_Hybrid")
        midas.to(device)
        midas.eval()
        midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
        transform = midas_transforms.dpt_transform
    return midas, transform

_ = load_midas()
print("✅ MiDaS depth model ready.")


In [ ]:

#@title ⬆️ Upload your image
from google.colab import files

uploaded = files.upload()
assert len(uploaded) > 0, "Please upload a photo."
image_path = list(uploaded.keys())[0]
print("📷 Image:", image_path)


In [ ]:

#@title 🛠 Helper functions: depth, parallax, grading

def read_image_cv2(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is None:
        raise RuntimeError("Failed to read image.")
    return img

@torch.no_grad()
def estimate_depth(img_bgr):
    midas, transform = load_midas()
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    input_img = Image.fromarray(img_rgb)
    input_batch = transform(input_img).to(device)
    prediction = midas(input_batch).squeeze().cpu().numpy()
    # normalize to [0,1]
    d = prediction
    d = (d - d.min()) / (d.max() - d.min() + 1e-6)
    return d

def apply_parallax(img, depth, shift_x=10, shift_y=6):
    """Simple depth-aware parallax using warping.
    Positive shift_x pans to the right (foreground moves more than background).
    """
    h, w = depth.shape
    yy, xx = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')
    # foreground (smaller depth) moves more → use (1 - depth)
    fg = 1.0 - depth
    map_x = (xx - fg * shift_x).astype(np.float32)
    map_y = (yy - fg * shift_y).astype(np.float32)
    warped = cv2.remap(img, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
    return warped

def smoothstep(edge0, edge1, x):
    t = np.clip((x - edge0) / (edge1 - edge0 + 1e-6), 0.0, 1.0)
    return t * t * (3 - 2 * t)

def cinematic_grade(img, intensity=0.18, lift=0.02, gamma=0.95, gain=1.12, vignette=0.35, glow=0.15):
    imgf = img.astype(np.float32) / 255.0
    # simple lift-gamma-gain
    graded = np.clip((imgf + lift), 0, 1)
    graded = np.power(graded, gamma)
    graded = np.clip(graded * gain, 0, 1)
    # gentle teal-orange split via channel bias
    bias = np.array([1.06, 1.0, 0.94], dtype=np.float32)  # BGR bias (cool shadows, warm highs)
    graded = np.clip(graded * bias, 0, 1)

    # vignette
    h, w = graded.shape[:2]
    y, x = np.ogrid[:h, :w]
    cx, cy = w / 2.0, h / 2.0
    rx, ry = w * 0.75, h * 0.75
    v = 1.0 - smoothstep(0.0, 1.0, ((x-cx)**2/(rx*rx) + (y-cy)**2/(ry*ry)))
    v = (1 - vignette) + vignette * v
    graded *= v[..., None]

    # glow (bloom)
    blur = cv2.GaussianBlur((graded*255).astype(np.uint8), (0,0), sigmaX=6, sigmaY=6)
    blur = blur.astype(np.float32)/255.0
    graded = np.clip(graded + glow * blur, 0, 1)

    # overall intensity mix
    out = (imgf*(1-intensity) + graded*intensity)
    return np.clip(out*255, 0, 255).astype(np.uint8)

def zoom_pan(img, t, zoom_start=1.0, zoom_end=1.12, pan_x=0.0, pan_y=0.0):
    """t in [0,1]"""
    h, w = img.shape[:2]
    z = zoom_start + (zoom_end - zoom_start)*t
    M = np.array([[z, 0, -w*(z-1)/2 + pan_x*t],
                  [0, z, -h*(z-1)/2 + pan_y*t]], dtype=np.float32)
    out = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
    return out


In [ ]:

#@title 🎞️ Generate premium 3‑clip cinematic sequence
#@markdown You can tweak durations (seconds) and fps if you like.
fps = 24 #@param {type:"slider", min:8, max:60, step:1}
clip1_sec = 4 #@param {type:"slider", min:2, max:8, step:1}
clip2_sec = 4 #@param {type:"slider", min:2, max:8, step:1}
clip3_sec = 4 #@param {type:"slider", min:2, max:8, step:1}

img = read_image_cv2(image_path)
h, w, _ = img.shape

# Estimate depth once
depth = estimate_depth(img)

frames = []

# Clip 1: Wide → gentle zoom-in with grading
for i in range(int(clip1_sec*fps)):
    t = i / max(1, (clip1_sec*fps - 1))
    f = zoom_pan(img, t, zoom_start=1.0, zoom_end=1.08, pan_x=8, pan_y=6)
    f = cinematic_grade(f, intensity=0.22, vignette=0.32, glow=0.12)
    frames.append(f)

# Clip 2: Parallax pan with lens glow
for i in range(int(clip2_sec*fps)):
    t = i / max(1, (clip2_sec*fps - 1))
    shift_x = 14 * (t - 0.5)  # left to right
    shift_y = 8 * math.sin(t*math.pi) * 0.5
    f = apply_parallax(img, depth, shift_x=shift_x, shift_y=shift_y)
    f = zoom_pan(f, t, zoom_start=1.02, zoom_end=1.10, pan_x=-10, pan_y=4)
    f = cinematic_grade(f, intensity=0.26, vignette=0.38, glow=0.18)
    frames.append(f)

# Clip 3: Close, slow zoom + soft blur + fade to black
for i in range(int(clip3_sec*fps)):
    t = i / max(1, (clip3_sec*fps - 1))
    f = zoom_pan(img, t, zoom_start=1.06, zoom_end=1.18, pan_x=4, pan_y=-6)
    # bokeh-like softness
    k = int(1 + 2* t * 2) * 2 + 1  # odd kernel grows slightly
    f = cv2.GaussianBlur(f, (k, k), sigmaX=1.2 + 1.8*t)
    f = cinematic_grade(f, intensity=0.28, vignette=0.42, glow=0.22)
    # fade to black
    fade = 1.0 - 0.8 * (t**1.5)
    f = np.clip(f * fade, 0, 255).astype(np.uint8)
    frames.append(f)

# Save video
video_path = "cinematic_video.mp4"
clip = ImageSequenceClip([cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames], fps=fps)
clip.write_videofile(video_path, codec="libx264", audio=False, verbose=False, logger=None)

print("✅ Done! Saved:", video_path)


In [ ]:

#@title ⬇️ Download the video
from google.colab import files
files.download("cinematic_video.mp4")
